In [112]:
# Identify athletes meeting new Dec 24 OCTC selection rules: 

#1. Segregate into Male and Femal 
#2. For each gender perform the following: 
#a. Sort data by mapped eent, then perf scalar (higher the better)
#b. Identify tiers based on performance - Tier 1 is meets bronze medal mark for SEAG, Tier 2 is 2% and Tier 3 is 3.5%
#c. Check - if athlete met bronze or 2%/3.5% then delta_benchmark is zero or +, delta2% is + and delta 3.5% is +
#d. Top ranked athletes for each event are chosen. Max number of athletes for each event is 3, except for 100m/400m which is 6
#    This includes athletes on spex scholarship and potential
#e. The max for each tier is 2. Lower ranked athletes move down one tier.
#3. If athlete qualifies for more than one event the higher tier event is given
#4. Jump and throws junior program to be solved separately

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2565]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy



from google.cloud import storage



Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.


In [4713]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4714]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [4715]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [4716]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [4717]:
competitors[competitors['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,None,None,+0.1,Sprint,Male,National School Games,07-04,2024,Local
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,None,None,None,Sprint,Male,National School Games,07-04,2024,Local
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,None,None,None,Sprint,Male,National School Games,07-04,2024,Local
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,None,None,None,Sprint,Male,National School Games,07-04,2024,Local
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,None,None,None,Sprint,Female,National School Games,07-04,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,None,None,+1.1,Sprint,Male,National School Games,11-04,2024,Local
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,None,None,None,Sprint,Female,National School Games,13-04,2024,Local
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,None,None,+0.2,Sprint,Female,National School Games,15-04,2024,Local
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,None,None,None,Jump,Male,National School Games,19-04,2024,Local


In [4718]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.loc[rowIndex,'DATE']
    year = competitors.loc[rowIndex,'YEAR']    
    
    if 'to' in date or ' - ' in date:
        
        pos = re.search('to|\s\-\s', date)
                                                
        # Splice string to day and month

        split_pos_start=pos.start()+3

            
        final_date = date[split_pos_start:] # left string post splicing
        final_year = year[2:]
        
        event_date = final_date + '/' + final_year
        
        competitors.loc[rowIndex, 'event_date'] = event_date
        
    elif re.search('\w\-\w', date):
        
        if competitors.iloc[rowIndex, 15] == "National School Games":
        
            event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd
        
            competitors.loc[rowIndex, 'event_date'] = event_date
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

In [4719]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='%d%m%Y', errors='coerce')

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['event_date_dt'].dt.month


In [4720]:
# Choose last 6 months or 12 months of results 

#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

#competitors=competitors.reset_index(drop=True)

#start = datetime.datetime(2024, 10, 22)

#end = datetime.datetime(2025, 3, 31)

#start_date = np.datetime64(start)
#end_date = np.datetime64(end)


#mask = (competitors['event_date_dt'] >= start_date) & (competitors['event_date_dt'] <= end_date)
#athletes_selected = competitors.loc[mask]

In [4721]:
athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [4722]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,NaN,NaN
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,NaN,NaN
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,9/7/24,NaT,NaT,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN


In [4723]:
athletes_selected[athletes_selected['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,Female,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,Male,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,NaN,NaN
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,Female,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,NaN,NaN
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,Female,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,NaN,NaN
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,Male,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,NaN,NaN


In [4724]:
athletes_selected.to_csv('athletes_selected_prod.csv', encoding='utf-8')

In [4725]:
# Choose 2024/25 only

athletes = athletes_selected

In [4726]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,NaN,NaN
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,NaN,NaN
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,9/7/24,NaT,NaT,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN


In [4727]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
1413,Jun Jie Calvin Quek,53.19,None,None,4,None,400m Hurdles,None,None,None,...,Male,UAE Athletics Grand Prix,3-May,2024,International,3-May-24,NaT,NaT,NaN,NaN
1414,Jun Jie Calvin Quek,52.05,None,None,8,None,400m Hurdles,None,None,None,...,Male,11th Kinami Michitaka Memorial Athletics Meet,12-May,2024,International,12-May-24,NaT,NaT,NaN,NaN
1516,Jun Jie Calvin Quek,51.8,None,None,1,None,400m Hurdles,None,None,None,...,Male,Taiwan Athletics Open,1 - 2 June,2024,International,2 June/24,NaT,NaT,NaN,NaN
1544,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,...,Male,XXXIII Qosanov Memorial,22 - 23 June,2024,International,23 June/24,NaT,NaT,NaN,NaN
1545,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,...,Male,XXXIII Qosanov Memorial,22 - 23 June,2024,International,23 June/24,NaT,NaT,NaN,NaN
1689,Jun Jie Calvin Quek,52.18,None,None,3,None,400m Hurdles,None,None,None,...,Male,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International,29 May/24,NaT,NaT,NaN,NaN
1690,Jun Jie Calvin Quek,51.33,None,None,2,None,400m Hurdles,None,None,None,...,Male,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International,29 May/24,NaT,NaT,NaN,NaN
24681,Jun Jie Calvin Quek,52.62,None,None,9,None,400m Hurdles,None,None,None,...,Male,Sydney Track Classic,15-Mar,2025,International,15-Mar-25,NaT,NaT,NaN,NaN


In [4728]:
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'80', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2435207703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''


In [4729]:
athletes[(athletes['COMPETITION']=='National School Games')]

#athletes[(athletes['EVENT']=='110m Hurdles')].tail(50)

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN,
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN,
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN,
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN,
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,NaN,NaN,
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,NaN,NaN,
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,NaN,NaN,
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,NaN,NaN,High Jump


In [4730]:
athletes[athletes['MAPPED_EVENT']=='Marathon']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
548,"HE, HENRY YONG",2:44:28,Singapore,41,20,None,Marathon,None,None,None,...,Asian Marathon,21-Jan,2024,International,21-Jan-24,NaT,NaT,NaN,NaN,Marathon
665,DEVATHAS SATIANATHAN?,2:56:03,INDIVIDUAL,-,None,None,Marathon,None,None,None,...,Boston Marathon (USA),15-Apr,2024,International,15-Apr-24,NaT,NaT,NaN,NaN,Marathon
666,"HIA, CALEB",2:43:40,INDIVIDUAL,-,190,None,Marathon,None,None,None,...,Chicago Marathon (USA),13-Oct,2024,International,13-Oct-24,NaT,NaT,NaN,NaN,Marathon
668,"TAN, YONG JIN, KERWIN",2:49:39,INDIVIDUAL,-,None,None,Marathon,None,None,None,...,Chicago Marathon (USA),13-Oct,2024,International,13-Oct-24,NaT,NaT,NaN,NaN,Marathon
669,ANSGAR CHENG,2:51:45,INDIVIDUAL,58,None,None,Marathon,None,None,None,...,Berlin Marathon (Germany),29-Sep,2024,International,29-Sep-24,NaT,NaT,NaN,NaN,Marathon
670,PU YE,3:22:38,INDIVIDUAL,54,None,None,Marathon,None,None,None,...,Tokyo Marathon (Japan),3-Mar,2024,International,3-Mar-24,NaT,NaT,NaN,NaN,Marathon
671,ALAN YUMING CAO,2:50:35,INDIVIDUAL,48,None,None,Marathon,None,None,None,...,Wuxi Marathon (China),24-Mar,2024,International,24-Mar-24,NaT,NaT,NaN,NaN,Marathon
672,CHENG HEOW WONG,2:58:53,INDIVIDUAL,45,None,None,Marathon,None,None,None,...,Seoul Marathon (Korea),17-Mar,2024,International,17-Mar-24,NaT,NaT,NaN,NaN,Marathon
673,YVONNE ELIZABETH CHEE,3:12:19,INDIVIDUAL,45,None,None,Marathon,None,None,None,...,NN Marathon Rotterdam (Netherlands),14-Apr,2024,International,14-Apr-24,NaT,NaT,NaN,NaN,Marathon
674,CHEE YONG ONG,2:56:16,INDIVIDUAL,44,None,None,Marathon,None,None,None,...,Seoul Marathon (Korea),17-Mar,2024,International,17-Mar-24,NaT,NaT,NaN,NaN,Marathon


In [4731]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:6: SettingWithCopyWarning: 
A 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:6: SettingWithCopyWarning: 
A 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.strip()
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [4732]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


athletes.to_csv('athletes_post_map_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [4733]:
#credentials = service_account.Credentials.from_service_account_file(
#    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
#)

#sql="""
#SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
#FROM `saa-analytics.results.saa_full`
#WHERE STAGE='Final' AND COMPETITION='SEA Game AND RANK='3'
#"""

#benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




In [4734]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT YEAR, EVENT, SUB_EVENT, GENDER, NAME, RESULT, RANK, CATEGORY_EVENT, COMPETITION, STAGE, HEAT
FROM `saa-analytics.benchmarks.saa_benchmarks_prod`
WHERE YEAR='2023' AND COMPETITION='Southeast Asian Games' AND (RANK='3' OR RANK='3.0')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4735]:
SEAG

,YEAR,EVENT,SUB_EVENT,GENDER,NAME,RESULT,RANK,CATEGORY_EVENT,COMPETITION,STAGE,HEAT
0,2023,4 x 100m,None,Male,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,Relay,Southeast Asian Games,None,None
1,2023,4 x 400m,None,Male,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,Relay,Southeast Asian Games,None,None
2,2023,4 x 100m,None,Female,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,Relay,Southeast Asian Games,None,None
3,2023,4 x 400m,None,Female,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,Relay,Southeast Asian Games,None,None
4,2023,400m,None,Male,Frederick Ramirez,46.63,3,Sprint,Southeast Asian Games,None,None
...,...,...,...,...,...,...,...,...,...,...,...
75,2023,100m Hurdles,None,Female,Natchaya Chowpakpang,14.23,3,Hurdles,Southeast Asian Games,Heats,Heat 2
76,2023,Marathon,None,Male,Nguyen Thanh Hoang,2:35:49,3.0,Marathon,Southeast Asian Games,None,None
77,2023,Marathon,None,Female,Christine Organiza Hallasgo,2:50:27,3.0,Marathon,Southeast Asian Games,None,None
78,2023,20km Race Walk,None,Female,Kotchaphon Tangsrivong,1:57:11,3.0,20km Race Walk,Southeast Asian Games,None,None


In [4736]:
SEAG_filtered=SEAG[SEAG['HEAT'].isnull() & SEAG['SUB_EVENT'].isnull()]

In [4737]:
benchmarks=SEAG_filtered

In [4738]:
benchmarks

,YEAR,EVENT,SUB_EVENT,GENDER,NAME,RESULT,RANK,CATEGORY_EVENT,COMPETITION,STAGE,HEAT
0,2023,4 x 100m,None,Male,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,Relay,Southeast Asian Games,None,None
1,2023,4 x 400m,None,Male,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,Relay,Southeast Asian Games,None,None
2,2023,4 x 100m,None,Female,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,Relay,Southeast Asian Games,None,None
3,2023,4 x 400m,None,Female,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,Relay,Southeast Asian Games,None,None
4,2023,400m,None,Male,Frederick Ramirez,46.63,3,Sprint,Southeast Asian Games,None,None
5,2023,"10,000m",None,Male,Than Htike Soe,31:25.5,3,Long,Southeast Asian Games,None,None
6,2023,3000m Steeplechase,None,Male,Pandu Sukarya,08:55.0,3,Steeple,Southeast Asian Games,None,None
7,2023,110m Hurdles,None,Male,John Cabang,13.85,3,Hurdles,Southeast Asian Games,None,None
8,2023,400m Hurdles,None,Male,Jun Jie Calvin Quek,50.75,3,Hurdles,Southeast Asian Games,None,None
9,2023,High Jump,None,Male,Farrell Glenn Felix,2.15,3,Jump,Southeast Asian Games,None,None


In [4739]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/127349024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [4740]:
benchmarks.drop(['YEAR', 'HEAT', 'NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2795758766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarks.drop(['YEAR', 'HEAT', 'NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [4741]:
benchmarks=benchmarks.reset_index(drop=True)

In [4770]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                # fix different formats
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                    print('10Km here')

                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        print('1.5Km here')
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                    
        
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [4743]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.loc[rowIndex,'EVENT']
    
        metric=df.loc[rowIndex, 'BENCHMARK']
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [4744]:
process_benchmarks(benchmarks)

0 4 x 100m 39.36
1 4 x 400m 188.8
2 4 x 100m 44.58
3 4 x 400m 219.3
4 400m 46.63
5 10,000m 1885.5
6 3000m Steeplechase 535.0
7 110m Hurdles 13.85
8 400m Hurdles 50.75
9 High Jump 2.15
10 Shot Put 17.3
11 400m 53.84
12 1500m 266.3
13 10,000m 2131.0
14 400m Hurdles 59.09
15 Triple Jump 13.46
16 200m 21.02
17 800m 113.9
18 5000m 883.4
19 Discus Throw 50.02
20 Javelin Throw 66.2
21 Decathlon 6891.0
22 800m 129.2
23 5000m 1033.6
24 3000m Steeplechase 660.9
25 High Jump 1.73
26 Discus Throw 45.08
27 Heptathlon 5253.0
28 100m 10.45
29 100m 10.45
30 1500m 239.4
31 Pole Vault 5.2
32 Long Jump 7.62
33 Triple Jump 15.7
34 Hammer Throw 59.76
35 100m 11.75
36 200m 23.6
37 100m Hurdles 13.59
38 Pole Vault 4.0
39 Long Jump 6.02
40 Shot Put 14.44
41 Hammer Throw 49.61
42 Javelin Throw 48.31
43 Marathon 9349.0
44 Marathon 10227.0
45 20km Race Walk 7031.0
46 20km Race Walk 6579.0


,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric
0,4 x 100m,None,Male,39.36,39.36
1,4 x 400m,None,Male,03:08.8,188.80
2,4 x 100m,None,Female,44.58,44.58
3,4 x 400m,None,Female,03:39.3,219.30
4,400m,None,Male,46.63,46.63
5,"10,000m",None,Male,31:25.5,1885.50
6,3000m Steeplechase,None,Male,08:55.0,535.00
7,110m Hurdles,None,Male,13.85,13.85
8,400m Hurdles,None,Male,50.75,50.75
9,High Jump,None,Male,2.15,2.15


In [4745]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.loc[rowIndex,'EVENT']
    
    metric=benchmarks.loc[rowIndex,'BENCHMARK']
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

0 4 x 100m 39.36
1 4 x 400m 188.8
2 4 x 100m 44.58
3 4 x 400m 219.3
4 400m 46.63
5 10,000m 1885.5
6 3000m Steeplechase 535.0
7 110m Hurdles 13.85
8 400m Hurdles 50.75
9 High Jump 2.15
10 Shot Put 17.3
11 400m 53.84
12 1500m 266.3
13 10,000m 2131.0
14 400m Hurdles 59.09
15 Triple Jump 13.46
16 200m 21.02
17 800m 113.9
18 5000m 883.4
19 Discus Throw 50.02
20 Javelin Throw 66.2
21 Decathlon 6891.0
22 800m 129.2
23 5000m 1033.6
24 3000m Steeplechase 660.9
25 High Jump 1.73
26 Discus Throw 45.08
27 Heptathlon 5253.0
28 100m 10.45
29 100m 10.45
30 1500m 239.4
31 Pole Vault 5.2
32 Long Jump 7.62
33 Triple Jump 15.7
34 Hammer Throw 59.76
35 100m 11.75
36 200m 23.6
37 100m Hurdles 13.59
38 Pole Vault 4.0
39 Long Jump 6.02
40 Shot Put 14.44
41 Hammer Throw 49.61
42 Javelin Throw 48.31
43 Marathon 9349.0
44 Marathon 10227.0
45 20km Race Walk 7031.0
46 20km Race Walk 6579.0


In [4746]:
benchmarks

,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric
0,4 x 100m,None,Male,39.36,39.36
1,4 x 400m,None,Male,03:08.8,188.80
2,4 x 100m,None,Female,44.58,44.58
3,4 x 400m,None,Female,03:39.3,219.30
4,400m,None,Male,46.63,46.63
5,"10,000m",None,Male,31:25.5,1885.50
6,3000m Steeplechase,None,Male,08:55.0,535.00
7,110m Hurdles,None,Male,13.85,13.85
8,400m Hurdles,None,Male,50.75,50.75
9,High Jump,None,Male,2.15,2.15


In [4747]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [4748]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT'].str.strip()

In [4749]:
for col in benchmarks.columns:
    benchmarks[col] = benchmarks[col].astype(str)
    benchmarks[col] = benchmarks[col].str.replace('\xa0', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\r', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\n', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.strip()


In [4750]:
benchmarks.head(50)

,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric,2%,3.5%,5%,MAPPED_EVENT
0,4 x 100m,None,Male,39.36,39.36,40.1472,40.73759999999999,41.328,4 x 100m
1,4 x 400m,None,Male,03:08.8,188.8,192.57600000000002,195.408,198.24,4 x 400m
2,4 x 100m,None,Female,44.58,44.58,45.4716,46.140299999999996,46.809,4 x 100m
3,4 x 400m,None,Female,03:39.3,219.3,223.686,226.97549999999998,230.26500000000001,4 x 400m
4,400m,None,Male,46.63,46.63,47.5626,48.26205,48.96150000000001,400m
5,"10,000m",None,Male,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775,"10,000m"
6,3000m Steeplechase,None,Male,08:55.0,535.0,545.7,553.7249999999999,561.75,3000m Steeplechase
7,110m Hurdles,None,Male,13.85,13.85,14.127,14.334749999999998,14.5425,110m Hurdles
8,400m Hurdles,None,Male,50.75,50.75,51.765,52.52625,53.2875,400m Hurdles
9,High Jump,None,Male,2.15,2.15,2.1069999999999998,2.07475,2.0425,High Jump


In [4751]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,nan,nan,Hammer Throw
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,nan,nan,Pole Vault
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,9/7/24,NaT,NaT,nan,nan,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,nan,nan,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,nan,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m


In [4752]:
athletes[athletes['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,nan,nan,
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,nan,nan,
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,nan,nan,
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,nan,nan,High Jump


In [4753]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [4754]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [4755]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,nan,nan,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,nan,nan,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328


In [4756]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,nan,nan,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45


In [4757]:
df[df['MAPPED_EVENT']=='Decathlon']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1601,Lucas Le Cong Fun,6431,None,None,1,U20,Decathlon,None,None,None,...,nan,nan,Decathlon,Decathlon,None,6891,6891.0,6753.18,6649.815,6546.45
26520,Jayden Ng,5716,None,None,3,None,Decathlon U18,None,None,None,...,nan,nan,Decathlon,Decathlon,None,6891,6891.0,6753.18,6649.815,6546.45


In [4758]:
# replace '-' with NaN

df['RESULT'] = df['RESULT'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [4759]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,nan,nan,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,nan,nan,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328


In [4760]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,nan,nan,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45


In [4761]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


df.to_csv('df_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [4762]:
for col in df.columns:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()


In [4771]:
# Convert results and seed into seconds format

df.reset_index(drop=True, inplace=True)


for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    event=df.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=df.loc[rowIndex,'RESULT'] # result
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='DNS' or metric=='NH' or metric=='NM' or metric=='FOUL' or metric=='DNF' or metric=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
    
    if event=='1500m':
        
        print(rowIndex, event, result, result_out)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out


8 1500m 07:00.5 420.5
26 1500m 04:23.9 263.9
43 1500m 04:57.0 297.0
74 1500m 05:39.7 339.7
75 1500m 06:50.5 410.5
76 1500m 06:57.1 417.1
101 1500m 07:38.1 458.1
116 1500m 07:48.1 468.1
178 1500m 07:00.5 420.5
269 1500m 05:31.8 331.8
295 1500m 06:21.0 381.0
318 1500m 05:00.6 300.6
343 1500m 06:03.4 363.4
345 1500m 06:19.2 379.2
357 1500m 05:32.0 332.0
367 1500m 05:38.8 338.8
393 1500m 06:17.6 377.6
494 1500m 08:01.9 481.9
497 1500m 07:02.1 422.1
527 1500m 05:42.7 342.7
534 1500m 04:08.6 248.6
545 1500m 04:24.5 264.5
582 1500m 04:51.7 291.7
587 1500m 04:22.7 262.7
595 1500m 04:25.4 265.4
598 1500m 04:43.4 283.4
599 1500m 04:47.3 287.3
10Km here
10Km here
10Km here
10Km here
623 1500m 05:24.1 324.1
626 1500m 06:28.0 388.0
670 1500m 04:52.1 292.1
673 1500m 08:31.2 511.2
716 1500m 04:46.6 286.6
717 1500m 04:53.8 293.8
1.5Km here
727 1500m 4:13:02 253.02
10Km here
778 1500m 04:18.8 258.8
779 1500m 04:19.0 259.0
783 1500m 04:46.1 286.1
785 1500m 04:59.4 299.4
806 1500m 04:20.2 260.2
816 1500m

14407 1500m 04:55.2 295.2
14419 1500m 07:47.6 467.6
14420 1500m 05:28.5 328.5
14431 1500m 05:58.9 358.9
14455 1500m 04:58.8 298.8
14467 1500m 07:06.2 426.2
14469 1500m 06:33.6 393.6
14470 1500m 07:02.2 422.2
14477 1500m 07:24.3 444.3
14517 1500m 05:05.0 305.0
14539 1500m 05:56.7 356.7
14540 1500m 05:54.3 354.3
14553 1500m 06:11.9 371.9
14555 1500m 05:48.4 348.4
14557 1500m 05:48.1 348.1
14562 1500m 06:27.9 387.9
14588 1500m 05:18.5 318.5
14606 1500m 04:36.5 276.5
14623 1500m 04:54.1 294.1
14625 1500m 04:42.1 282.1
14626 1500m 04:34.4 274.4
14679 1500m 06:33.6 393.6
14684 1500m 05:15.2 315.2
14685 1500m 07:00.1 420.1
14687 1500m 06:11.2 371.2
14688 1500m 05:57.2 357.2
14691 1500m 05:51.6 351.6
14692 1500m 05:53.5 353.5
14693 1500m 06:21.0 381.0
14694 1500m 06:10.2 370.2
14697 1500m 05:36.7 336.7
14738 1500m 05:35.1 335.1
14746 1500m 05:39.3 339.3
14776 1500m 04:56.4 296.4
14780 1500m 06:14.1 374.1
14783 1500m 06:10.2 370.2
14786 1500m 06:00.0 360.0
14792 1500m 06:02.5 362.5
14812 1500m 

19438 1500m 04:17.2 257.2
19439 1500m 04:20.1 260.1
19445 1500m 05:24.9 324.9
19449 1500m 04:41.6 281.6
19456 1500m 04:49.0 289.0
19458 1500m 04:56.1 296.1
19477 1500m 04:08.4 248.4
19484 1500m 04:14.1 254.1
19488 1500m 05:42.5 342.5
19490 1500m 04:38.5 278.5
19491 1500m 05:29.4 329.4
19493 1500m 04:28.0 268.0
19517 1500m 04:36.3 276.3
19518 1500m 04:57.1 297.1
19522 1500m 04:48.2 288.2
19541 1500m 04:47.9 287.9
19544 1500m 04:53.0 293.0
19553 1500m 05:02.2 302.2
19584 1500m 04:28.0 268.0
19601 1500m 05:15.1 315.1
19604 1500m 05:22.6 322.6
19612 1500m 05:18.6 318.6
19613 1500m 04:49.0 289.0
19652 1500m 04:15.7 255.7
19712 1500m 04:26.6 266.6
19728 1500m 04:16.5 256.5
19733 1500m 05:29.3 329.3
19734 1500m 05:46.3 346.3
20406 1500m 06:16.5 376.5
20407 1500m 06:54.9 414.9
20408 1500m 06:38.9 398.9
20409 1500m 06:17.8 377.8
20410 1500m 05:23.7 323.7
20411 1500m 06:27.7 387.7
20412 1500m 05:57.7 357.7
20413 1500m 05:17.6 317.6
20414 1500m 06:09.3 369.3
20415 1500m 06:04.7 364.7
20416 1500m 

26491 1500m 04:03.1 243.1
26521 1500m 04:08.7 248.7
26524 1500m 04:03.9 243.9
26525 1500m 03:59.6 239.6


In [4653]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [4654]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,nan,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772,32.4
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,nan,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995,4.41
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06


In [4655]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [4656]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772,32.4,32.4
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995,4.41,4.41
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06


In [4657]:
df[df['MAPPED_EVENT']=='1500m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
8,"Moey, Bernie",07:00.5,SMA,76,1,Masters,Run,1500,,nan,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,420.5,420.5
26,"Ong, Marcus",04:23.9,UNA,38,1,Masters,Run,1500,,nan,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,263.9,263.9
43,"Kato, Takao",04:57.0,UNA,51,1,Masters,Run,1500,,nan,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,297.0,297.0
74,"Tan, Mervyn",05:39.7,MR25,46,2,Masters,Run,1500,,nan,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,339.7,339.7
75,"., Matthew",06:50.5,MR25,60,2,Masters,Run,1500,,nan,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,410.5,410.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24455,"Su, Emma",05:05.6,Hwa Chong Institution College,18,1,Open,Run,1500,None,E839C07,...,1500m,1500m,None,04:26.3,266.3,271.62600000000003,275.6205,279.615,305.6,305.6
26491,Oliver Lim,04:03.1,None,None,9,None,1500m,None,None,None,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,243.1,243.1
26521,Amir Rusyaidi Osman,04:08.7,None,None,11,None,1500m,None,None,None,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,248.7,248.7
26524,Ethan Yan,04:03.9,None,None,2,None,1500m,None,None,None,...,1500m,1500m,None,03:59.4,239.4,244.18800000000002,247.779,251.37,243.9,243.9


In [4658]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric)

In [4659]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|hetathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='Southeast Asian Games'] # Do not include results from SEAG in dataset

In [4660]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [4661]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


df.to_csv('df_test_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [4662]:
df[df['MAPPED_EVENT']=='10,000m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
540,"Goh, Shaun",32:34.2,Club ZOOM,27,1,Open,Run,10000,,nan,...,NaN,NaN,NaN,1954.2,1954.2,NaN,NaN,NaN,NaN,NaN
604,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,1923.21,1951.4925,1979.775,2274.0,2274.0,-350.79,-322.5075,-294.225,-388.5,-15.604614
605,LEE VANESSA YING ZHUANG,35:55:00,SINGAPORE,26,8,None,10000m,None,None,None,...,2173.62,2205.5850,2237.550,2155.0,2155.0,18.62,50.5850,82.550,-24.0,3.873768
606,VANESSA LEE YING ZHUANG,40:21:00,SINGAPORE,26,10,None,10000m,None,None,None,...,2173.62,2205.5850,2237.550,2421.0,2421.0,-247.38,-215.4150,-183.450,-290.0,-8.608634
611,"GOH, SHAUN",33:48:00,Singapore,27,87,None,10000m,None,None,None,...,1923.21,1951.4925,1979.775,2028.0,2028.0,-104.79,-76.5075,-48.225,-142.5,-2.557677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24543,"Ahmed, Nawaz",39:49.8,Lacticbuds,22,11,Open,Run,10000,None,N034I03,...,1923.21,1951.4925,1979.775,2389.8,2389.8,-466.59,-438.3075,-410.025,-504.3,-21.746221
24544,"Ko, Keane",33:51.9,Club ZOOM,25,1,Open,Run,10000,None,K326A00,...,1923.21,1951.4925,1979.775,2031.9,2031.9,-108.69,-80.4075,-52.125,-146.4,-2.764519
24581,"NG, YEW CHEO",48:27.8,Cougars Athletic Association,39,2,Open,Run,10000,None,Y971Z86,...,2173.62,2205.5850,2237.550,2907.8,2907.8,-734.18,-702.2150,-670.250,-776.8,-31.452370
24582,"Teoh, Katelyn",49:09.4,National University Singapore,20,3,Open,Run,10000,None,K831W05,...,2173.62,2205.5850,2237.550,2949.4,2949.4,-775.78,-743.8150,-711.850,-818.4,-33.404505


In [4663]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4664]:
# Read name variations from GCS name lists bucket (Still in beta)


df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")


In [4665]:
df_names=df[((df['NAME']=='Heng, Richard')|(df['NAME']=='Marican, Shohib'))]

In [4666]:
df_names.to_csv("names.csv", encoding='utf-8')

In [4667]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [4668]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4669]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4670]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [4671]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian
239,King,Leonard


In [4672]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [4673]:
no_foreigners_list

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27848,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27849,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27850,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
27851,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4674]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [4675]:
# Choose the best result for each event participated by every athlete and reset index

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

top_performers_clean.reset_index(drop=True, inplace=True)


In [4676]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


top_performers_clean.to_csv('top_performers_clean_prod.csv', encoding='utf-8')

In [4677]:
top_performers_clean

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,H132F05,...,15.3860,15.1505,14.915,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,"ooi, chen si",9.22m,Ngee Ann Polytechnic,18,9,Open,Triple Jump,0,,nan,...,13.1908,12.9889,12.787,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,"chua, keira",10.16m,Cedar Girls Secondary School,16,5,U18,Triple Jump,0,,nan,...,13.1908,12.9889,12.787,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,J034B11,...,15.3860,15.1505,14.915,10.47,10.47,-4.9160,-4.6805,-4.445,-5.23,-28.312102
4,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,nan,...,15.3860,15.1505,14.915,11.67,11.67,-3.7160,-3.4805,-3.245,-4.03,-20.668790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15872,"., Mohd Farrel",31.28,Seng Kang Primary School,10,38,U11,Dash,150,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
15873,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
15874,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
15875,"., Asher",11.76,Erovra Club,6,16,7-Aug,Dash,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [4678]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [4679]:
tiered_performers

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,H132F05,...,15.3860,15.1505,14.915,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,"ooi, chen si",9.22m,Ngee Ann Polytechnic,18,9,Open,Triple Jump,0,,nan,...,13.1908,12.9889,12.787,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,"chua, keira",10.16m,Cedar Girls Secondary School,16,5,U18,Triple Jump,0,,nan,...,13.1908,12.9889,12.787,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,J034B11,...,15.3860,15.1505,14.915,10.47,10.47,-4.9160,-4.6805,-4.445,-5.23,-28.312102
4,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,nan,...,15.3860,15.1505,14.915,11.67,11.67,-3.7160,-3.4805,-3.245,-4.03,-20.668790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15872,"., Mohd Farrel",31.28,Seng Kang Primary School,10,38,U11,Dash,150,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
15873,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
15874,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
15875,"., Asher",11.76,Erovra Club,6,16,7-Aug,Dash,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [4680]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [4681]:
tiered_performers[tiered_performers['MAPPED_EVENT']=='1500m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
7515,"wee, Nicholas",04:30.9,Lacticbuds,18,1,U20,Run,1500,,nan,...,247.7790,251.370,270.9,270.9,-26.712,-23.1210,-19.530,-31.5,-8.157895,
7516,"wee, Micole",05:36.3,Lacticbuds,15,2,U18,Run,1500,,nan,...,275.6205,279.615,336.3,336.3,-64.674,-60.6795,-56.685,-70.0,-21.286143,
7517,"teoh, Claudia",05:58.6,Total Kinetics Sports,15,5,U18,Run,1500,,,...,275.6205,279.615,358.6,358.6,-86.974,-82.9795,-78.985,-92.3,-29.660158,
7518,"teo, swee lin",05:25.0,TeamFabian,15,4,U18,Run,1500,None,S312F10,...,275.6205,279.615,325.0,325.0,-53.374,-49.3795,-45.385,-58.7,-17.042809,
7519,"tay, zixiang",04:45.1,TEMASEK POLYTECHNIC,20,18,Open,Run,1500,,nan,...,247.7790,251.370,285.1,285.1,-40.912,-37.3210,-33.730,-45.7,-14.089390,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8315,"ANDREA, TANG TZE TING (DENG",07:25.8,Chij St Theresa's Convent,16,26,U18,Run,1500,None,T165E09,...,275.6205,279.615,445.8,445.8,-174.174,-170.1795,-166.185,-179.5,-62.405182,
8316,"AMEDREZA D'EZEQUIEL, BIN SOPHIAN",06:02.4,Northbrooks Secondary School,14,22,U15,Run,1500,None,B586E11,...,247.7790,251.370,362.4,362.4,-118.212,-114.6210,-111.030,-123.0,-46.378446,
8317,"ALKATHIB, ANISAH",05:51.3,Victoria Junior College,17,1,U20,Run,1500,,nan,...,275.6205,279.615,351.3,351.3,-79.674,-75.6795,-71.685,-85.0,-26.918888,
8318,"ABDULLAH, ABDUL HADI",04:46.0,Dunman High School,18,5,U20,Run,1500,None,A914C07,...,247.7790,251.370,286.0,286.0,-41.812,-38.2210,-34.630,-46.6,-14.465330,


In [4682]:
# Drop rows without a SEAG benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [4683]:
# Process dates to extract age

# Map NSG divisions into age

mask = (final_df['DIVISION'].str.contains(r'A', na=False))
final_df.loc[mask, 'AGE'] = '18.5'

mask = (final_df['DIVISION'].str.contains(r'B', na=False))
final_df.loc[mask, 'AGE'] = '16'

mask = (final_df['DIVISION'].str.contains(r'C', na=False))
final_df.loc[mask, 'AGE'] = '13.5'

mask = (final_df['DIVISION'].str.contains(r'O', na=False))
final_df.loc[mask, 'AGE'] = '12'



In [4684]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


final_df['DOB_new'] = final_df['DOB'].apply(length)



#B = parser.parse("10-09-2021", dayfirst = True)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/3934348418.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = final_df['DOB'].apply(length)


In [4685]:
final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')

final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:5: SettingWithCopyWarning: 
A value is 

In [4686]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1795310803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract']=final_df['age_extract'].apply(age)


In [4687]:
# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/4177143290.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


In [4688]:
# Change to numeric

final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1263148356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)


In [4689]:
final_df[final_df['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
8,Yvette Lee,10.13,VJC,18.5,7,A,Triple Jump,None,None,None,...,10.13,-3.0608,-2.85890,-2.6570,-3.33,-19.739970,,NaT,NaN,18.5
10,Yuan Yinze,11.54,CH,16,15,B,Triple Jump,None,None,None,...,11.54,-3.8460,-3.61050,-3.3750,-4.16,-21.496815,,NaT,NaN,16.0
11,Yong Zeheng,9.96,SJI,13.5,15,C,Triple Jump,None,None,None,...,9.96,-5.4260,-5.19050,-4.9550,-5.74,-31.560510,,NaT,NaN,13.5
13,"Yeow Kai Xin, Charis",9.26,ACJC,18.5,17,A,Triple Jump,None,None,None,...,9.26,-3.9308,-3.72890,-3.5270,-4.20,-26.203566,,NaT,NaN,18.5
17,Yen Ming Zhen,11.93,NJC,16,9,B,Triple Jump,None,None,None,...,11.93,-3.4560,-3.22050,-2.9850,-3.77,-19.012739,,NaT,NaN,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8375,"Ho, Joanna",00:18.00,RI,18.5,6,A,100m Hurdles,None,0.840m,None,...,18.00,-4.1382,-3.93435,-3.7305,-4.41,-27.450331,,NaT,NaN,18.5
8377,Chloe Loh,00:20.98,HCI,18.5,11,A,100m Hurdles,None,0.840m,None,...,20.98,-7.1182,-6.91435,-6.7105,-7.39,-49.378219,,NaT,NaN,18.5
8382,Angela Li Yu Qing,00:17.02,SSP,18.5,3,A,100m Hurdles,None,0.840m,None,...,17.02,-3.1582,-2.95435,-2.7505,-3.43,-20.239146,,NaT,NaN,18.5
8384,"Ang Yu Xi, Sophie",00:17.54,RI,18.5,4,A,100m Hurdles,None,0.840m,None,...,17.54,-3.6782,-3.47435,-3.2705,-3.95,-24.065489,,NaT,NaN,18.5


In [4690]:
final_df=final_df[((final_df['COMPETITION']!='National School Games')|((final_df['COMPETITION']=='National School Games') & (final_df['MAPPED_EVENT']!='100m Hurdles') & (final_df['MAPPED_EVENT']!='400m Hurdles') & (final_df['MAPPED_EVENT']!='Shot Put') & (final_df['MAPPED_EVENT']!='Javelin Throw') & (final_df['MAPPED_EVENT']!='Hammer Throw') & (final_df['MAPPED_EVENT']!='Discus Throw')))]

In [4691]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


final_df.to_csv('tiered_performers_prod.csv', encoding='utf-8')

In [4692]:
# Rank everyone for published ranking lists

published_ranking = final_df.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
published_ranking['Rank'] = published_ranking.groupby(['GENDER', 'MAPPED_EVENT']).cumcount() + 1

published_ranking.to_csv('published_ranking_prod.csv', encoding='utf-8')

In [4383]:
# Convert time format for marathon and 5000m into mm:ss.00
# Choose the correct column indices or you will get erratic timings

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

all_ranking=all_ranking.reset_index(drop=True)


#all_ranking[['2%', '3.5%', '5%']] = df[['2%', '3.5%', '5%']].apply(pd.to_numeric)


#all_ranking['2%'] = all_ranking['2%'].astype("string")
#all_ranking['3.5%'] = all_ranking['3.5%'].astype("string")
#all_ranking['5%'] = all_ranking['5%'].astype("string")


for i in range(len(all_ranking)):
        
    rowIndex = all_ranking.index[i]

    event=all_ranking.iloc[rowIndex,21]
        
    
    time_base2=all_ranking.iloc[rowIndex,25]
    time_base3=all_ranking.iloc[rowIndex,26]
    time_base5=all_ranking.iloc[rowIndex,27]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10,000m' or event=='5000m' or event=='3000m Steeplechase' or event=='1500m':
        
      #  print(i, event, time_base2, time_base3, time_base5)

        
        

            
        
        date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
        date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
        date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)
        
    #    print(date_preconvert2, date_preconvert3, date_preconvert5)
            
        
        output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
        output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
        output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
     #   print(event, output2, output3, output5)

                    
       #     top_performers_clean.loc[rowIndex, '2%_timing'] = output2
       #     top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
       #     top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
   
        all_ranking.at[rowIndex, '2%'] = output2 # copy over time format
        all_ranking.at[rowIndex, '3.5%'] = output3
        all_ranking.at[rowIndex, '5%'] = output5


            


        
    elif event=='Marathon':
        
      #  print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
        #    top_performers_clean.loc[rowIndex, '2%_timing'] = output2
        #    top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
        #    top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            all_ranking.at[rowIndex, '2%'] = output2 # copy over time format
            all_ranking.at[rowIndex, '3.5%'] = output3
            all_ranking.at[rowIndex, '5%'] = output5

            
         #   print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             
